In [1]:
import sys
sys.path.insert(1, 'D:/kaneshi')
import kaneshi

import os
from os import listdir
from os.path import isfile, join

import dash
from dash import html, dcc
from dash.dependencies import Output, Input

from kaneshi.trading.visual import ClearRSIPlot
from kaneshi.config import ROOT_DIR

In [4]:
default_dir = f'{ROOT_DIR}/trading/databases/'

config = {
    'database_name': 'clear_rsi_XRPUSDT_30_50_55',
    'bottom_edge': 50,
    'upper_edge': 55,
    'show_last_n': 500,
    'time_step': 's',
    'chart_shift': 50,
}

In [5]:
def get_layout(app, options):
    app.layout = html.Div(
        [dcc.Dropdown(options=options,
                      id='db_dropdown',
                      value=options[0],
                      multi=False,
                      ),

         dcc.Graph(id='live-graph',
                   animate=True,
                   config=dict({'scrollZoom': True})),

         dcc.Interval(
             id='graph-update',
             interval=2_000,
             n_intervals=0
         ),
        ]
    )
    return app


def get_files(path: str = None):
    files = [f for f in listdir(path) if isfile(join(path, f))]
    return [os.path.splitext(file)[0] for file in files]


app = dash.Dash(__name__)
app = get_layout(app, get_files(default_dir))


@app.callback(
    Output('live-graph', 'figure'),
    [Input('graph-update', 'n_intervals'),
     Input('db_dropdown', 'value')]
)
def update_graph_scatter(n, db_name):
    plotter = ClearRSIPlot(**config)

    fig = plotter.plot()
    fig.update_layout(
        dragmode='pan',
        newshape_line_color='cyan',
        modebar_add=['drawline',
                     'eraseshape'])
    return fig

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
